# FABRIC Example: Map of Active Resources


## Configure the Environment

In [130]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

#Configure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
beta-2.fabric-testbed.net


### Save the Initial Refresh Token

In [131]:
#Retrieve or set the refresh token (exprires 24 hours after login)
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTKNRVGRRTEYJZMI4DINJZMM2DANJYMMYWMMDFG5STQNLFGM4WMMB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGIYDEMJSGM4DSNJGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTIZBYGY2TIMDCMIYWINTFGJTDAN3DGFSTKZRVMQZTEMZQMJTDQOJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGE4DIOBTGA4TMNRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


## Create and Initialize Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [132]:
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name = 'all', scope = 'all')
slice_manager.initialize()

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [133]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTOMLCMI3DANZYMJSDGMBWMNSWEYTCG43GKYRQGEYTSYJUGIZTGMZ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRYGIYDIMZUG44TEMBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)


### Query Active Slices

In [134]:
status, slices = slice_manager.slices(state = "All")
active_slices = []

from time import gmtime, strftime
now = strftime("%Y-%m-%d %H:%M:%S", gmtime())

print("Response Status {}".format(status))
if status == Status.OK:
    for slice in slices:
        if slice.slice_state != 'Dead' and (now < slice.lease_end):
            active_slices.append(slice)
            print(slice)
else:
    print(f"Failure: {slices}")

Response Status Status.OK
{
    "graph_id": "788d3fee-914f-4c85-9bca-315d418c1f44",
    "lease_end": "2021-08-06 20:44:19",
    "slice_id": "82b26488-8973-47b0-a04c-5b425b19e230",
    "slice_name": "Slice-sts-ded",
    "slice_state": "Closing"
}
{
    "graph_id": "8d472954-dd99-498b-b6a0-6252c81905b9",
    "lease_end": "2021-08-06 21:09:26",
    "slice_id": "4a15718b-19e9-4eb1-af19-ac302d9e5189",
    "slice_name": "Slice-l2bridge-sriov",
    "slice_state": "StableOK"
}


### Get IDs of Active Slices, Query Slivers From Slices

In [135]:
slice_ids = [slice.slice_id for slice in active_slices]
slice_names = [slice.slice_name for slice in active_slices]

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)
    
    print(f'\nSlice Name: {slice_names[slice_ids.index(slice_id)]}\n')
    print("Response Status {}".format(status))
    if status == Status.OK:
        print("Slivers {}".format(slivers))
    else:
        print(f"Failure: {slivers}")


Slice Name: Slice-sts-ded

Response Status Status.OK
Slivers [{
    "graph_node_id": "f6f9f5ed-46f4-4950-a201-1beb95ce8b59",
    "join_state": "NoJoin",
    "lease_end": "2021-08-06 20:44:19",
    "name": "sts1",
    "pending_state": "None_",
    "reservation_id": "909f6686-ad8b-4092-a057-63a4d4e0b573",
    "reservation_state": "Closed",
    "resource_type": "L2STS",
    "slice_id": "82b26488-8973-47b0-a04c-5b425b19e230"
}, {
    "allocated_capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "allocated_labels": "{\"instance_parent\": \"uky-w2.fabric-testbed.net\"}",
    "capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 6}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d10\"}",
    "graph_node_id": "6a3a092b-5fd8-476e-8abb-a26e565a6bed",
    "join_state": "NoJoin",
    "lease_end": "2021-08-06 20:44:19",
    "name": "n1",
    "pending_state": "None_",
    "reservation_id": "fa6535a7-f137-481a-b424-dd7131fb1bd2",
    "reservation_state": "Closed",
    "resource_ty

### Create and Save Map

- Click radio buttons to generate map for different slices
- Click markers to view details about individual resources
- Map saves as interactable HTML file

In [136]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENC': (35.93996154497003, -79.01809264307425), 'LBNL': (38.94977311613871, -122.61289106281062)}

import folium
import branca
from folium.plugins import MarkerCluster
from folium.features import DivIcon

base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2)

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)

    labels = {}
    for sliver in slivers:
        status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
        if sliver_status.resource_type == "VM":
            name = sliver_status.name
            site = sliver_status.site
            capacities = sliver_status.capacities
            info = (site, capacities, site_dict[site])
            labels[name] = info
            sliver_status.

    locations = []
    popups = []
    icons = []

    for label in labels:
        locations.append(site_dict[labels[label][0]])
        popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Verdana">Resource: {label}</strong><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}', width = 300, height = 70), max_width = 300))
        icons.append(folium.Icon(color = 'green', icon = 'server', prefix = 'fa'))
        
    cluster = MarkerCluster(name = slice_names[slice_ids.index(slice_id)], 
                            locations = locations, 
                            popups = popups, 
                            icons = icons,
                            show = False)
    folium.map.Marker((52, -110), 
                      icon = DivIcon(icon_size = (300, 60),
                                     icon_anchor = (0, 0),
                                     html = f'<div style="font-size: 20pt;font-family: Verdana">{slice_names[slice_ids.index(slice_id)]}</div>')).add_to(cluster)
    cluster.add_to(base)
    
    
    
                                                 
folium.LayerControl(collapsed = False).add_to(base)
folium.map.Marker((54, -75), 
                      icon = DivIcon(icon_size = (110, 60),
                                     icon_anchor = (0, 0),
                                     html = '<p style="font-size: 10pt;font-family: Verdana;text-align: center">Click to generate map for resource</p>')).add_to(base)

base.save('map.html')
base